In [29]:
import time
import requests
import json

current_milli_time = lambda: int(round(time.time() * 1000))
locale_mappings={'en':'en_GB', 'ru':'ru_Nothing',
                 'es':'es_Nothing', 'fr':'fr_Nothing',
                 'de':'de_Nothing', 'ja':'ja_Nothing',
                 'zh':'zh_CN'
                }
def query_duckling(text, lang):
    if lang in locale_mappings:
        locale=locale_mappings[lang]
    else:
        return {'result':'fail', 'cause':"unsupport lang"}
    data={'locale':locale, 'text':text, 'reftime':current_milli_time()}
    response = requests.post('http://0.0.0.0:8000/parse', data=data)
    if response.status_code == 200:
        r=response.json()
        # print(json.dumps(r, indent=2, ensure_ascii=False))
        return {'result':'success', 'data':r}
    return {'result':'fail', 'cause':'error response'}

resp=query_duckling('tomorrow at eight', 'en')
resp

{'data': [{'body': 'tomorrow at eight',
   'dim': 'time',
   'end': 17,
   'latent': False,
   'start': 0,
   'value': {'grain': 'hour',
    'type': 'value',
    'value': '2019-08-02T08:00:00.000-07:00',
    'values': [{'grain': 'hour',
      'type': 'value',
      'value': '2019-08-02T08:00:00.000-07:00'},
     {'grain': 'hour',
      'type': 'value',
      'value': '2019-08-02T20:00:00.000-07:00'}]}}],
 'result': 'success'}

In [8]:
def query_entities(data):
    response = requests.post('http://localhost:8092/entities', json=data)
    if response.status_code == 200:
        r=response.json()
        return {'result':'success', 'data':r}
    return {'result':'fail', 'cause':'error response'}
ents=query_entities({"lang":"en", "sents":"I am from China"})
ents

{'data': [{'end': 15, 'entity': 'GPE', 'start': 10, 'text': 'China'}],
 'result': 'success'}

In [9]:
[d['entity'] for d in ents['data']]

['GPE']

In [14]:
[d['dim'] for d in resp['data']]

['time']

In [25]:
import requests
from sagas.nlu.patterns import Patterns, print_result, Inspector

class NegativeWordInspector(Inspector):        
    def name(self):
        return "ins_negative_word"

    def run(self, key, ctx):
        result=False
        # domains=dispatcher.domains
        if ctx.meta['lang']=='da':
            if 'ikke' in ctx.chunks[key] or 'ikke'==ctx.lemmas[key]:
                result=True
        return result

class DateInspector(Inspector):     
    def __init__(self, dim):
        self.dim=dim
        
    def name(self):
        return "ins_date"

    def run(self, key, ctx):
        result=False
        lang=ctx.meta['lang']
        cnt=' '.join(ctx.chunks[key])
        print('date ins ->', cnt)
        resp=query_duckling(cnt, lang)
        if resp['result']=='success':
            if self.dim in [d['dim'] for d in resp['data']]:
                result=True
        return result

class EntityInspector(Inspector):     
    def __init__(self, dim):
        self.dim=dim
        
    def name(self):
        return "ins_entity"

    def run(self, key, ctx):
        result=False
        lang=ctx.meta['lang']
        cnt=' '.join(ctx.chunks[key])
        resp=query_entities({'lang':lang, 'sents':cnt})
        if resp['result']=='success':
            dims=[d['entity'] for d in resp['data']]
            print('entities ->', ', '.join(dims))
            if self.dim in dims:
                print('%s ∈'%cnt, self.dim)
                result=True
        return result

def procs(data):
    response = requests.post('http://localhost:8090/verb_domains', json=data)
    rs = response.json()
    r=rs[0]
    print(r)
    domains=r['domains']
    meta={'rel':r['rel'], **data}
    agency=['c_pron', 'c_noun']
    rs=[Patterns(domains, meta, 1).verb(nsubj=agency, obj=agency),
        Patterns(domains, meta, 2).verb(nsubj=agency, obj=agency, advmod=NegativeWordInspector()),
        Patterns(domains, meta, 2).verb(nsubj_pass=agency, obl=DateInspector('time')),
        Patterns(domains, meta, 2).verb(nsubj_pass=agency, obl=EntityInspector('GPE')),
       ]
    print_result(rs)

text='Han har ikke tøjet på.'
data = {'lang': 'da', "sents": text}
procs(data)

{'type': 'verb_domains', 'verb': 'har', 'index': '2', 'rel': 'root', 'governor': 0, 'domains': [['nsubj', '1', 'Han', 'han', ['Han'], ['c_pron']], ['advmod', '3', 'ikke', 'ikke', ['ikke'], ['c_adv']], ['obj', '4', 'tøjet', 'tøj', ['tøjet'], ['c_noun']], ['compound:prt', '5', 'på', 'på', ['på'], ['c_adv']], ['punct', '6', '.', '.', ['.'], ['c_punct']]]}
✔ [verb with nsubj is ['c_pron', 'c_noun']: True, obj is ['c_pron', 'c_noun']: True]
✔ [verb with nsubj is ['c_pron', 'c_noun']: True, obj is ['c_pron', 'c_noun']: True, advmod is Inspector('ins_negative_word'): True]
✖ [verb with nsubj:pass is ['c_pron', 'c_noun']: False, obl is Inspector('ins_date'): False]
✖ [verb with nsubj:pass is ['c_pron', 'c_noun']: False, obl is Inspector('ins_entity'): False]


In [23]:
# text='I was born in the spring of 1982.'
text='I was born in Beijing in the spring of 1982.'
data = {'lang': 'en', "sents": text}
procs(data)

{'type': 'verb_domains', 'verb': 'born', 'index': '3', 'rel': 'root', 'governor': 0, 'domains': [['nsubj:pass', '1', 'I', 'I', ['I'], ['c_pron', 'x_prp']], ['aux:pass', '2', 'was', 'be', ['was'], ['c_aux', 'x_vbd']], ['obl', '5', 'Beijing', 'Beijing', ['in', 'Beijing'], ['c_propn', 'x_nnp']], ['obl', '8', 'spring', 'spring', ['in', 'the', 'spring', 'of', '1982'], ['c_noun', 'x_nn']], ['punct', '11', '.', '.', ['.'], ['c_punct', 'x_.']]]}
date ins -> in the spring of 1982
entities DATE
✖ [verb with nsubj is ['c_pron', 'c_noun']: False, obj is ['c_pron', 'c_noun']: False]
✖ [verb with nsubj is ['c_pron', 'c_noun']: False, obj is ['c_pron', 'c_noun']: False, advmod is Inspector('ins_negative_word'): False]
✔ [verb with nsubj:pass is ['c_pron', 'c_noun']: True, obl is Inspector('ins_date'): True]
✖ [verb with nsubj:pass is ['c_pron', 'c_noun']: True, obl is Inspector('ins_entity'): False]


In [28]:
text='I was born in Beijing.'
data = {'lang': 'en', "sents": text}
procs(data)

{'type': 'verb_domains', 'verb': 'born', 'index': '3', 'rel': 'root', 'governor': 0, 'domains': [['nsubj:pass', '1', 'I', 'I', ['I'], ['c_pron', 'x_prp']], ['aux:pass', '2', 'was', 'be', ['was'], ['c_aux', 'x_vbd']], ['obl', '5', 'Beijing', 'Beijing', ['in', 'Beijing'], ['c_propn', 'x_nnp']], ['punct', '6', '.', '.', ['.'], ['c_punct', 'x_.']]]}
date ins -> in Beijing
entities -> GPE
in Beijing ∈ GPE
✖ [verb with nsubj is ['c_pron', 'c_noun']: False, obj is ['c_pron', 'c_noun']: False]
✖ [verb with nsubj is ['c_pron', 'c_noun']: False, obj is ['c_pron', 'c_noun']: False, advmod is Inspector('ins_negative_word'): False]
✖ [verb with nsubj:pass is ['c_pron', 'c_noun']: True, obl is Inspector('ins_date'): False]
✔ [verb with nsubj:pass is ['c_pron', 'c_noun']: True, obl is Inspector('ins_entity'): True]


In [10]:
x={'y':6}
r={'x':5, **x}
r

{'x': 5, 'y': 6}

In [38]:
def trip_number_suffix(k):
    if k[-2]=='_' and k[-1].isdigit():
        return k[:-2]
    return k
def service_method(*args, **kwargs):
    print(kwargs)
    for k,v in kwargs.items():
        # if k[-2]=='_' and k[-1].isdigit():
        #    print(k[:-2],v)
        print(trip_number_suffix(k), v)
service_method(x_x=5, x_2=6)

{'x_x': 5, 'x_2': 6}
x_x 5
x 6
